In [1]:
#https://pypi.org/project/transformers/
from transformers import pipeline
import haystack
import transformers 

/Users/abaygi/miniforge3/envs/mytransfenv/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
haystack.__version__

'1.10.0'

In [3]:
transformers.__version__

'4.21.2'

In [ ]:
#After running the Docker Desktop run the following command in terminal
#docker run -p 9200:9200 -e "discovery.type=single-node" docker.elastic.co/elasticsearch/elasticsearch:7.14.2

In [4]:
## create an instantiate of ElasticsearchDocumentStore
from haystack.document_stores import ElasticsearchDocumentStore
document_store = ElasticsearchDocumentStore(host="localhost", username="", password="", index="document")


/Users/abaygi/miniforge3/envs/mytransfenv/lib/python3.9/site-packages/elasticsearch/connection/base.py:208: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.14/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


In [5]:
# Let's first fetch some documents that we want to query
import pandas as pd 
df = pd.read_csv('../cleaned_data/FINAL_CORD_DATA.csv')

In [6]:
df.head(3)

,sha,title,abstract,publish_time,authors,url,body_text
0,41ed6b3014e89604b351c095571c45056ba12c37,A phantom study to optimise the automatic tube current modulation for chest ...,"On March 11, 2020, the World Health Organization declared the coronavirus di...",2021-05-28,"Gombolevskiy, Victor; Morozov, Sergey; Chernina, Valeria; Blokhin, Ivan; Vas...",https://doi.org/10.1186/s41747-021-00218-0; https://www.ncbi.nlm.nih.gov/pub...,\nWe obtained a density difference (ground-glass opacities minus visually un...
1,57221999055bdc0b1c4bb16c790fdebdac1f7ce7,"Tunicamycin, an anticancer drug and inhibitor of N- linked glycosylation pro...","SARS-CoV-2 outbreaks remains a medical and economic challenge, due to the la...",2020-10-20,"Dawood, Ali Adel; Altobje, Mahmood Abduljabar",https://doi.org/10.1016/j.micpath.2020.104586; https://www.ncbi.nlm.nih.gov/...,Tunicamycin is an antibiotic was produced by Streptomyces clavuligerus and S...
2,7fd64a56bf3a761da96e6aad00789413813599ca,Asymptomatic SARS-CoV-2 infection,NaN,2020-06-12,"Ooi, Eng Eong; Low, Jenny G",https://doi.org/10.1016/s1473-3099(20)30460-6; https://api.elsevier.com/cont...,The pandemic spread of severe acute respiratory syndrome coronavirus-2 (SARS...


In [7]:
df.shape

(24910, 7)

In [8]:
df.isnull().sum()

sha                0
title              0
abstract        3067
publish_time       0
authors          218
url                0
body_text          0
dtype: int64

In [9]:
df.dropna(inplace=True)

In [10]:
df.isnull().sum()

sha             0
title           0
abstract        0
publish_time    0
authors         0
url             0
body_text       0
dtype: int64

In [11]:
# Convert files to dicts
# You can optionally supply a cleaning function that is applied to each doc (e.g. to remove footers)
# It must take a str as input, and return a str.
dicts = df.to_dict('records')

In [12]:
#dicts[3]

In [13]:
dicts[3]["sha"]

'fa5968c0d0290201f8eb1ca2eaf98181ae6a7a6f'

In [14]:
dicts[3]["title"]

'Mechanical Ventilator Parameter Estimation for Lung Health through Machine Learning'

In [15]:
#dicts[3]["body_text"]

### 
from haystack.document_stores import ElasticsearchDocumentStore



document_store = ElasticsearchDocumentStore()


documents = [
    Document(
    
      'content'=DOCUMENT_TEXT_HERE
      
      'meta'={'name': DOCUMENT_NAME, ...}
      
    ),
    ...
]



document_store.write_documents(dicts)

In [16]:
final_dicts = []
for each in dicts:
    tmp = {}
    if each is not None:
        tmp['content'] = each.pop('body_text')
        tmp['meta'] = each
    final_dicts.append(tmp)

document_store.write_documents(final_dicts)

/Users/abaygi/miniforge3/envs/mytransfenv/lib/python3.9/site-packages/elasticsearch/connection/base.py:208: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.14/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


In [34]:
#final_dicts[0]

In [ ]:
#document_store.write_documents(final_dicts)

In [ ]:
#test_dicts = [ {
        #'content': "hi",
       # 'meta': {'name': "DOCUMENT_NAME"}
   # }]

In [ ]:
#document_store.write_documents(test_dicts)

In [18]:
document_store.get_document_count()

21696

### How to build the QA System

In [21]:
import fastapi
import pydantic

In [22]:
fastapi.__version__

'0.88.0'

In [23]:
pydantic.__version__

'1.10.2'

In [24]:
from fastapi import FastAPI
from pydantic import BaseModel

from haystack.document_stores import ElasticsearchDocumentStore
import os
from haystack.pipelines import ExtractiveQAPipeline
from haystack.nodes import FARMReader
from haystack.nodes import BM25Retriever

In [25]:
ELASTIC_SEARCH_HOST =  os.environ.get('es_ip', 'localhost')
ELASTIC_SEARCH_PORT =  os.environ.get('es_port', 9200)

In [26]:
ELASTIC_SEARCH_HOST

'localhost'

In [27]:
#docker run -p 9200:9200 -e "discovery.type=single-node" docker.elastic.co/elasticsearch/elasticsearch:7.14.2
document_store = ElasticsearchDocumentStore(host=ELASTIC_SEARCH_HOST,
                                            port=ELASTIC_SEARCH_PORT,
                                            username="", 
                                            password="",
                                            index="document")



### Initialize Retriever, Reader & Pipeline

In [28]:
retriever = BM25Retriever(document_store=document_store)
reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2-covid", use_gpu=False)
pipe = ExtractiveQAPipeline(reader, retriever)

In [29]:
question = "what are the symptoms of covid"
k_retriver = 1
k_reader = 1
prediction = pipe.run(query=question, params={"Retriever": {"top_k": k_retriver}, "Reader": {"top_k": k_reader}})


Inferencing Samples:   0%|                                                                            | 0/1 [00:00<?, ? Batches/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Inferencing Samples: 100%|████████████████████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.33s/ Batches]


In [33]:
#prediction

In [31]:
len(prediction["answers"])

1

In [32]:
prediction["answers"][0]

<Answer {'answer': 'fever, cough,', 'type': 'extractive', 'score': 0.7558503150939941, 'context': 'p between the initial symptoms of COVID-19 (i.e., the three symptoms fever, cough, and shortness of breath that were identified by CDC during the firs', 'offsets_in_document': [{'start': 10297, 'end': 10310}], 'offsets_in_context': [{'start': 69, 'end': 82}], 'document_id': '59e0d84800d89ac46a83986bda699776', 'meta': {'sha': '9a94d7e76c6608825f0a81f66f94b2add6902a10; 2a6734a919f141b57591b3dd4b5290ac00f268d2', 'title': 'Miscommunication in the age of communication: A crowdsourcing framework for symptom surveillance at the time of pandemics', 'abstract': 'OBJECTIVE: There was a significant delay in compiling a complete list of the symptoms of COVID-19 during the 2020 outbreak of the disease. When there is little information about the symptoms of a novel disease, interventions to contain the spread of the disease would be suboptimal because people experiencing symptoms that are not yet known